# Introduction

* Python 3.5 and later have built-in support for *coroutines*. 
* Coroutines allow for concurrent threads of execution within a single process.
* Coroutines are cheap both in terms of CPU time and memory
  - Can run *lots* of them
* Good model for I/O bound code
* Similar to generators but with some restrictions.
* Coroutines are co-operative
  - A coroutine will keep running until it explictly yields control
* Similar/related:
  - Twisted
  - Tornado
  - gevent
  - Stackless Python
  - generators and `yield from`

In [1]:
async def foo():
    print("hello")
    
c = foo()
print(c)

<coroutine object foo at 0x7f97043e9d58>


In [2]:
next(c)

TypeError: 'coroutine' object is not an iterator

In [3]:
c.send(None)

hello


StopIteration: 

# Returning Values

In [4]:
async def foo():
    print("running")
    return "hello"

c = foo()
c.send(None)


running


StopIteration: hello

# Chaining Coroutines

In [5]:
async def double(x):
    return x * 2

async def compute(x, y):
    return await double(x) + y

async def print_result():
    result = await compute(2, 3)
    print("result: ", result)
    
try:
    print_result().send(None)
except StopIteration:
    pass


result:  7


# asyncio

* standard library package that has a bunch of tools for building asynchronous programs
* builds on async and await support in Python 3.5+
* also provides tools for doing similar things in older Pythons without async and await
  - using generators
  - available in stdlib from Python 3.4 onwards
  - available from PyPI for Python 3.3
* provides an *event loop*

In [6]:
import asyncio

async def hello():
    print("Hello!")
    
loop = asyncio.get_event_loop()
loop.run_until_complete(hello())


Hello!


# Running Multiple Coroutines

In [7]:
import asyncio

async def sleepy(t):
    print("sleeping", t)
    await asyncio.sleep(t)
    print("awake", t)
        
async def main():
    await asyncio.wait([sleepy(3), sleepy(1), sleepy(2)])      
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

sleeping 1
sleeping 3
sleeping 2
awake 1
awake 2
awake 3


In [8]:
import asyncio

async def sleepy(t):
    await asyncio.sleep(t)
    return t
    
async def main():
    pending = [sleepy(3), sleepy(1), sleepy(2)]
    done, _ = await asyncio.wait(pending)
    for f in done:
        print(await f)
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

3
1
2


In [10]:
import asyncio
from concurrent.futures import FIRST_COMPLETED

async def sleepy(t):
    await asyncio.sleep(t)
    return t
    
async def main():
    pending = [sleepy(3), sleepy(1), sleepy(2)]
    while pending:
        done, pending = await asyncio.wait(pending, return_when=FIRST_COMPLETED)      
        for f in done:
            print(await f)
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())


1
2
3


# Producer Consumer

Some new things here:
- interaction between coroutines using asyncio.Queue
- cancellation of tasks (much easier than with threads)

# Producer Consumer

Some new things here:
- interaction between coroutines using asyncio.Queue
- cancellation of tasks (much easier than with threads)

In [11]:
import asyncio
from concurrent.futures import CancelledError

async def producer(q):
    for i in range(20):
        await q.put(i)
        print("producer put {}".format(i))
    # wait for everything to be processed
    await q.join() 


async def consumer(name, q):
    print("consumer {} running".format(name))
    while True:
        try:
            value = await q.get()
        except CancelledError:
            print("consumer {} cancelled".format(name))
            return
        print("consumer {} got {}".format(name, value))
        q.task_done()
        

async def main():
    q = asyncio.Queue(1)
    
    # start some consumers
    consumers = asyncio.gather(
            consumer("A", q),
            consumer("B", q),
            consumer("C", q),
        )
    asyncio.ensure_future(consumers)
    
    # start the producer and wait for it finish
    await producer(q)
    
    # all done - tell the consumers to stop
    consumers.cancel()
  
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())


producer put 0
consumer A running
consumer A got 0
consumer C running
consumer B running
producer put 1
consumer A got 1
producer put 2
consumer C got 2
producer put 3
consumer B got 3
producer put 4
consumer A got 4
producer put 5
consumer C got 5
producer put 6
consumer B got 6
producer put 7
consumer A got 7
producer put 8
consumer C got 8
producer put 9
consumer B got 9
producer put 10
consumer A got 10
producer put 11
consumer C got 11
producer put 12
consumer B got 12
producer put 13
consumer A got 13
producer put 14
consumer C got 14
producer put 15
consumer B got 15
producer put 16
consumer A got 16
producer put 17
consumer C got 17
producer put 18
consumer B got 18
producer put 19
consumer A got 19
consumer A cancelled
consumer B cancelled
consumer C cancelled


# I/O

### Server

In [12]:
import asyncio

async def handle_echo(reader, writer):
    while True:
        line = await reader.readline()
        if not line:
            break
        writer.write(line)
        await writer.drain()


loop = asyncio.get_event_loop()
coro = asyncio.start_server(handle_echo, "127.0.0.1")
server = loop.run_until_complete(coro)
port = server.sockets[0].getsockname()[-1]
print('Serving on {}'.format(port))


Serving on 34949


### Client

In [13]:
async def echo_client(message):   
    reader, writer = await asyncio.open_connection('127.0.0.1', port)
    for i in range(4):
        writer.write(message.encode() + b"\n")
        line = await reader.readline()
        print('{!r} [{}]'.format(line.decode().strip(), i))
    writer.close()


async def run_clients():
    await asyncio.wait([
        echo_client("oxford"),
        echo_client("stilton"),
        echo_client("kikorangi"),
        echo_client("danish"),
        echo_client("cheddar"),
        echo_client("roquefort"),
        echo_client("gorgonzola"),
    ])
   

loop.run_until_complete(run_clients())

'cheddar' [0]
'roquefort' [0]
'gorgonzola' [0]
'stilton' [0]
'danish' [0]
'kikorangi' [0]
'cheddar' [1]
'roquefort' [1]
'oxford' [0]
'gorgonzola' [1]
'stilton' [1]
'danish' [1]
'kikorangi' [1]
'cheddar' [2]
'roquefort' [2]
'oxford' [1]
'gorgonzola' [2]
'stilton' [2]
'danish' [2]
'kikorangi' [2]
'cheddar' [3]
'roquefort' [3]
'oxford' [2]
'gorgonzola' [3]
'stilton' [3]
'danish' [3]
'kikorangi' [3]
'oxford' [3]


In [14]:
# Close the server
server.close()
loop.run_until_complete(server.wait_closed())

## Other I/O Types

* Aside from socket I/O asyncio has helpers for:
  - signal handling
  - subprocesses
* There's 3rd party library for other uses:
  - HTTP client and server: aiohttp
  - Files: aiofiles
  - and many more (search "aio" on PyPI)

# Alternatives to the asyncio package

* Apart from being in the standard library, there is nothing special about asyncio
* Anyone can built on async and await to built their own event loops etc
* There are plenty of complaints about asyncio being too big and complex
  - There's a lot of stuff in there!
* David Beazley's Curio is worth a look
  - https://github.com/dabeaz/curio
  - Has built-in support for async file I/O